# Summarize large text

Anton Antonov   
August 2025

----

## Introduction

This notebook illustrates how to specify a Large Language Model (LLM) graph for deriving comprehensive summaries of large texts. 
The LLM graph is based on different LLM- and non-LLM functions .
The Raku package ["LLM::Graph"](https://raku.land/zef:antononcube/LLM::Graph) is used, [AAp1].

Using the LLM graph is an alternative to the Literate programming based solutions shown in [AA1, AAn1].

----

## Setup

In [1]:
use LLM::Graph;
use Data::Importers;

use LLM::Tooling;

In [15]:
sink my $engine = 'neato';

In [16]:
#sink my $conf = llm-configuration('ChatGPT', model => 'gpt-5-mini')
sink my $conf = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', temperature => 0.55, max-tokens => 4096)

----

## LLM graph

Specify the LLM graph nodes:

In [3]:
sink my %rules =
TypeOfInput => sub ($_) {
        "Determine the input type of\n\n$_.\n\nThe result should be one of: 'Text', 'URL', 'FilePath', or 'Other'."  ~ 
        llm-prompt('NothingElse')('single string')
    },

IngestText =>  { eval-function => sub ($TypeOfInput, $_) { $TypeOfInput ~~ / URL | FilePath/ ?? data-import($_) !! $_} },

Title => { 
    eval-function => sub ($IngestText, $with-title = Whatever) { $with-title ~~ Str:D ?? $with-title !! llm-synthesize(llm-prompt("TitleSuggest")($IngestText, 'article')) },
},

Summary => sub ($IngestText) { llm-prompt("Summarize")() ~ "\n\n$IngestText" },

TopicsTable => sub ($IngestText) { llm-prompt("ThemeTableJSON")($IngestText, 'article', 20) },

ThinkingHats => sub ($IngestText) { llm-prompt("ThinkingHatsFeedback")($IngestText, 'HTML') },

MindMap => sub ($IngestText) { 
    [
        "Create a concise mermaid-js mind-map for the text:",
        $IngestText,
        "All node messages should be in quotes.",
        llm-prompt("NothingElse")("correct mermaid-js")
    ]
},

Report => { eval-function => 
    sub ($Title, $Summary, $TopicsTable, $MindMap, $ThinkingHats) { 
        [
            "# $Title",
            '### *LLM summary report*',
            '## Summary',
            $Summary,
            '## Topics',
            to-html(
                from-json($TopicsTable.subst(/ ^ '```json' | '```' $/):g),
                field-names => <theme content>,
                align => 'left'),
            "## Mind map",
            $MindMap,
            '## Thinking hats',
            $ThinkingHats.subst(/ ^ '```html' | '```' $/):g
        ].join("\n\n")
    } 
},

ExportAndOpen => {
    eval-function => sub ($Report) {
       spurt('./Report.md', $Report);
       shell "open ./Report.md" 
    },
    test-function => -> $export-and-open = True { $export-and-open ~~ Bool:D && $export-and-open || $export-and-open.Str.lc ∈ <true yes open> }
}
;

Make the graph:

In [5]:
my $gCombinedSummary = LLM::Graph.new(%rules, llm-evaluator => $conf)

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

----

## Full computation

URL and text statistics:

In [1]:
my $url = 'https://raw.githubusercontent.com/antononcube/RakuForPrediction-blog/refs/heads/main/Data/Graph-neat-examples-in-Raku-Set-3-YouTube.txt';
my $txtFocus = data-import($url);

text-stats($txtFocus)

(chars => 5861 words => 1158 lines => 156)

Computation:

In [7]:
$gCombinedSummary.eval($url, with-title => '«Graph» neat examples, set 3')

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

**Remark:** Instead of deriving the title using an LLM, the title is specified as an argument.

Show the corresponding graph:

In [ ]:
#% html
$gCombinedSummary.dot(node-width => 1.6, edge-color => 'DimGrey', edge-width => 0.8):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
 
 ThinkingHats_Cluster 
 
 
 
 Summary_Cluster 
 
 
 
 TopicsTable_Cluster 
 
 
 
 TypeOfInput_Cluster 
 
 
 
 MindMap_Cluster 
 
 
<!-- $_ -->
 
 $_ 
 
 $_ 
 
<!-- IngestText -->
 
 IngestText 
 
 IngestText 
 
<!-- $_->IngestText -->
 
 $_->IngestText 
 
 
 
<!-- TypeOfInput -->
 
 TypeOfInput 
 
 TypeOfInput 
 
<!-- $_->TypeOfInput -->
 
 $_->TypeOfInput 
 
 
 
<!-- ExportAndOpen -->
 
 ExportAndOpen 
 
 ExportAndOpen 
 
<!-- MindMap -->
 
 MindMap 
 
 MindMap 
 
<!-- IngestText->MindMap -->
 
 IngestText->MindMap 
 
 
 
<!-- Summary -->
 
 Summary 
 
 Summary 
 
<!-- IngestText->Summary -->
 
 IngestText->Summary 
 
 
 
<!-- ThinkingHats -->
 
 ThinkingHats 
 
 ThinkingHats 
 
<!-- IngestText->ThinkingHats -->
 
 IngestText->ThinkingHats 
 
 
 
<!-- Title -->
 
 Title 
 
 Title 
 
<!-- IngestText->Title -->
 
 IngestText->Title 
 
 
 
<!-- TopicsTable -->
 
 TopicsTable 
 
 TopicsTable 
 
<!-- IngestText->TopicsTable -->
 
 IngestText->TopicsTable 
 
 
 
<!-- Report -->
 
 Report 
 
 Report 
 
<!-- MindMap->Report -->
 
 MindMap->Report 
 
 
 
<!-- Report->ExportAndOpen -->
 
 Report->ExportAndOpen 
 
 
 
<!-- Summary->Report -->
 
 Summary->Report 
 
 
 
<!-- ThinkingHats->Report -->
 
 ThinkingHats->Report 
 
 
 
<!-- Title->Report -->
 
 Title->Report 
 
 
 
<!-- TopicsTable->Report -->
 
 TopicsTable->Report 
 
 
 
<!-- TypeOfInput->IngestText -->
 
 TypeOfInput->IngestText 
 
 
 
<!-- with-title -->
 
 with-title 
 
 with-title 
 
<!-- with-title->Title -->
 
 with-title->Title

Final result:

In [ ]:
#% markdown
$gCombinedSummary.nodes<Report><result>.subst(/'```html' | '```' $/):g

---

## Partial evaluation

Drop the results in `LLM::Graph` computed above:

In [11]:
$gCombinedSummary.drop-result

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

Here the are normalized nodes without results:

In [13]:
.say for |$gCombinedSummary.nodes

ThinkingHats => {eval-function => sub { }, input => [IngestText], spec-type => (Routine), test-function-input => [], wrapper => Routine::WrapHandle.new}
Summary => {eval-function => sub { }, input => [IngestText], spec-type => (Routine), test-function-input => [], wrapper => Routine::WrapHandle.new}
Report => {eval-function => sub { }, input => [Summary MindMap Title ThinkingHats TopicsTable], spec-type => (Callable), test-function-input => []}
TopicsTable => {eval-function => sub { }, input => [IngestText], spec-type => (Routine), test-function-input => [], wrapper => Routine::WrapHandle.new}
TypeOfInput => {eval-function => sub { }, input => [$_], spec-type => (Routine), test-function-input => [], wrapper => Routine::WrapHandle.new}
MindMap => {eval-function => sub { }, input => [IngestText], spec-type => (Routine), test-function-input => [], wrapper => Routine::WrapHandle.new}
ExportAndOpen => {eval-function => sub { }, input => [Report], spec-type => (Callable), test-function => ->

Here all results are pre-assigned as arguments:

In [14]:
$gCombinedSummary.eval($url, 
with-title => '«Graph» neat examples, set 3', 
:export-and-open,
Summary => 'In brief',
TypeOfInput => 'Other',
IngestText => 'Ingest text',
ThinkingHats => 'Thinking hats',
TopicsTable => '["TopicsTable"]',
MindMap => 'Mind-map graph',
)

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

-----

## References

### Blog posts

[AA1] Anton Antonov,
["Parameterized Literate Programming"](https://rakuforprediction.wordpress.com/2025/06/21/parameterized-literate-programming/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Notebooks

[AAn1] Anton Antonov,
["LLM comprehensive summary template for large texts"](https://community.wolfram.com/groups/-/m/t/3448842),
(2025),
[Wolfram Community](https://community.wolfram.com).

### Packages

[AAp1] Anton Antonov, 
[LLM::Graph, Raku package](https://github.com/antononcube/Raku-LLM-Graph),
(2025),
[GitHub/antononcube](https://github.com/antononcube).